In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/NicoZenith/Transformers.git

!pip install transformers
%cd Transformers

Mounted at /content/drive
Cloning into 'Transformers'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 63 (delta 29), reused 28 (delta 7), pack-reused 0
Receiving objects: 100% (63/63), 799.13 KiB | 15.37 MiB/s, done.
Resolving deltas: 100% (29/29), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.4 MB/s eta 0:00:00
/content/Transformers


In [2]:
import torch
import torch.nn as nn

import numpy as np

from transformers import AutoTokenizer
from transformer_layers import Head

from torch.nn import functional as F

In [3]:
with open('beatles.txt', 'r') as file:
    text = file.read()

print(text[:400])
print("length of dataset in characters", len(text))

A day in the life

I read the news today oh boy
About a lucky man who made the grade
And though the news was rather sad
Well I just had to laugh
I saw the photograph.

He blew his mind out in a car
He didn’t notice that the lights had changed
A crowd of people stood and stared
They'd seen his face before
Nobody was really sure
If he was from the House of Lords.

I saw a film today oh boy
The Engli
length of dataset in characters 168899


In [4]:
use_tokenizer = True
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

if use_tokenizer:
    # create a mapping from text to word tokens
    model_name = "bert-base-uncased"  # Replace with the desired pre-trained model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # Add a custom rule to handle line breaks
    text = text.replace('\n', '[')
    encode = lambda s: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s))
    decode = lambda l: tokenizer.convert_tokens_to_string([token.replace('[', '\n') if token == '[' \
                                                           else token for token in tokenizer.convert_ids_to_tokens(l)])
    vocab_size = tokenizer.vocab_size
else:
    # create a mapping from characters to integers
    stoi = { ch:i for i,ch in enumerate(chars) }
    itos = { i:ch for i,ch in enumerate(chars) }
    encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
    decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
    vocab_size = len(stoi)



# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print(train_data)


Token indices sequence length is longer than the specified maximum sequence length for this model (51158 > 512). Running this sequence through the model will result in indexing errors


tensor([1037, 2154, 1999,  ..., 1521, 1055, 1037])


In [5]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TextDataset(Dataset):
    def __init__(self, text, max_len):
        self.inputs = []
        self.labels = []
        for i in range(len(text) - max_len):
            x = text[i:i+max_len]
            y = text[i+1:i+max_len + 1]
            self.inputs.append(x)
            self.labels.append(y)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx]), torch.tensor(self.labels[idx])


batch_size = 8
max_len = 8
num_layers = 3
dropout = 0.2


train_dataset = TextDataset(train_data, max_len)
val_dataset = TextDataset(val_data, max_len)


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

x,y = next(iter(train_dataloader))

print(x[0])
print(y[0])




tensor([ 2001,  2189,  1998,  6919, 10529,  1031,  2027,  2425])
tensor([ 2189,  1998,  6919, 10529,  1031,  2027,  2425,  2033])


<ipython-input-5-cd0044c166cc>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.inputs[idx]), torch.tensor(self.labels[idx])


In [ ]:
class Cortical_Transformer(nn.Module):
    def __init__(self, d_model, dropout, max_len, vocab_size, num_layers, device):
        super().__init__()
        self.device = device
        self.max_len = max_len
        self.token_embedding_table = nn.Embedding(vocab_size, d_model)
        self.position_embedding_table = nn.Embedding(max_len, d_model)
        self.lm_head = nn.Linear(d_model, vocab_size)

        self.blocks = nn.Sequential(*[Head(d_model, d_model, dropout) for _ in range(num_layers)])
        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def compute_loss(self, logits, targets):
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        return loss


    def forward(self, idx, forward_mask=True):
        """
        Takes a context sequence of tokens as input, and generates the next token. Can be used only for prediction (targets = None)
        or for training.
        :idx: context sequence of tokens
        :valid_lens: actual length of the sequence
        :forward_mask: for unidirectional attention
        :targets: batch of targets for the next token
        """
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, d_model)
        pos_emb = self.position_embedding_table(torch.arange(T, device=self.device))
        x = tok_emb + pos_emb # (B, T, d_model)
        for blk in self.blocks:
            x = blk(x, valid_lens=None, forward_mask=forward_mask)
        logits = self.lm_head(x)

        return logits

        # if targets is None:
        #     loss = None
        # else:
        #     B, T, C = logits.shape
        #     logits = logits.view(B*T, C)
        #     targets = targets.view(B*T)
        #     loss = F.cross_entropy(logits, targets)

    def generate(self, idx, max_new_tokens, valid_lens=None, forward_mask=True):
        """
        Generate text by taking a context of size max_len and predicting the next token
        by sampling from the probability distribution obtained with the softmax on the whole vocabulary
        :idx: (B, T) array of indices in the current context
        :max_new_tokens: the number of new tokens we want to generate
        :return: generated sequence including the initial context
        """

        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.max_len:]
            # get the predictions
            logits = self(idx_cond, valid_lens, forward_mask)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim =1) # (B, T+1)
        return idx


In [ ]:
d_model = 24
num_epochs = 5
learning_rate = 1e-4
eval_interval = 50

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Cortical_Transformer(d_model, dropout, max_len, vocab_size, num_layers, device)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

for e in range(num_epochs):
    store_train_losses = []
    for i, batch in enumerate(train_dataloader):
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        model.zero_grad()

        logits = model(inputs)
        loss = model.compute_loss(logits, targets)
        loss.backward()
        optimizer.step()

        store_train_losses.append(loss.item())



1.500954 M parameters
10.326214790344238
10.326069831848145
10.325997352600098
10.325920104980469
10.32588005065918
10.325739860534668
10.32557487487793


/var/folders/jc/7gnlgyl53rdgpl4xt8pmy5h00000gn/T/ipykernel_1971/3213664333.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.inputs[idx]), torch.tensor(self.labels[idx])


10.325586318969727
10.325353622436523
10.325216293334961
10.325057983398438
10.325010299682617
10.324612617492676
10.324649810791016
10.32469654083252
10.324751853942871
10.324103355407715
10.324271202087402
10.324298858642578
10.324178695678711
10.323789596557617
10.323657989501953
10.323579788208008
10.32386589050293
10.323046684265137
10.323143005371094
10.322896003723145
10.322835922241211
10.322806358337402
10.322572708129883
10.322823524475098
10.322527885437012
10.322369575500488
10.322732925415039
10.32174301147461
10.321621894836426
10.3214693069458
10.321123123168945
10.321540832519531
10.321383476257324
10.320950508117676
10.321797370910645
10.321141242980957
10.320781707763672
10.321009635925293
10.3204927444458
10.32067584991455
10.320082664489746
10.320014953613281
10.320517539978027
10.319365501403809
10.319541931152344
10.319236755371094
10.31891918182373
10.31902027130127
10.319466590881348
10.3187894821167
10.31855297088623
10.318493843078613
10.31809139251709
10.3187

KeyboardInterrupt: 